In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig

In [2]:
model_name = "IlyaGusev/fred_t5_ru_turbo_alpaca"

generation_config = GenerationConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(50365, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50365, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [3]:
def get_summarize(text):
    sample = f"Задание: Суммаризируй текст, сохряняя важные детали.\nДано: {text}"
    data = tokenizer(sample, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    #torch.cuda.empty_cache()
    return tokenizer.decode(output_ids.tolist())

In [20]:
def get_summarize(text):
    sample = f"Задание: краткое содержание:\nДано: {text}"
    data = tokenizer(sample, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    #torch.cuda.empty_cache()
    return tokenizer.decode(output_ids.tolist())

In [4]:
def get_sum_df(old_df):
    df = old_df.copy()
    columns = ["Описание"]
    
    for i in tqdm(range(len(df))):
        for column in columns:
            text = df.iloc[i][column]
            if type(text) == str:
                
                #if len(text) > 12000:
                #    text = text[:6000]
                #    print("=============")
                #    print(i)
                #    print(column)
                #    print(len(text))
                #    print("=============")
                    
                df.iloc[i][column] = get_summarize(text)
    
    return df

In [5]:
df = pd.read_csv("atlas_raw.csv")

In [7]:
part1 = df.iloc[:100].copy()
new_part1 = get_sum_df(part1)
new_part1.to_csv("part1.1.csv")

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 100/100 [07:56<00:00,  4.76s/it]


In [8]:
part2 = df.iloc[100:].copy()
new_part2 = get_sum_df(part2)
new_part2.to_csv("part2.2.csv")

  2%|▏         | 2/93 [00:19<15:48, 10.43s/it]

2
Описание
6000


100%|██████████| 93/93 [09:08<00:00,  5.89s/it]


In [20]:
print(part2.iloc[45]["Описание"])

Описание. Маловетвистый колючий кустарник 

с многочисленными стволами (до 25 и более), высотой 
1,5-2, реже до 4-5 м. Корневая система расположена 
в верхнем слое почвы. Корневище сильно разветвлён-
ное, цилиндрическое, упругое, диаметром 0,6-3 (очень 
редко 4-5) см, с придаточными корнями, наиболее мно-
гочисленными в зоне выхода надземных побегов на 
поверхность. У хорошо развитых кустов общая длина 
корневища с корнями иногда достигает 30 м. Побеги 
прямые, со светло-серой корой, густо усажены, особен-
но в нижней части, игловидными, вниз отклоненными, 
ломкими, до 1 см длиной шипами, реже — без шипов. 
Листья черешковые, длина черешка 8-13 см. Листо-
вая пластинка округло-пятиугольная, до 15 см длиной 
и шириной; пальчатосложная, с 5-ю листочками, на 
черешочках 7-15 мм длиной; листочки эллиптические 
или обратноланцетные, с клиновидным основанием 
и узкой, довольно длинной оттянутой верхушкой, двоя-
козубчатые, с обеих сторон зеленые, опушенные корот-
кими, снизу по жилкам более 

In [21]:
print(new_part2.iloc[45]["Описание"])

<s>Выходные данные: Маловетвистый кустарник с многочисленными 
стволами (до 25 и более), высотой до 4-5 м. Корневище 
сильно разветвленное, цилиндрическое, упругое, диаметром 
0,6-3 (очень редко 4-5) см, с придаточными корнями, наиболее 
многочисленными в зоне выхода надземных побегов на 
поверхность. Листья черешковые, длина черешка 8-13 см. 
Листочки эллиптические или обратноланцетные, с 
клиновидным основанием и узкой, довольно длинной 
оттянутой верхушкой, двояко-козубчатые, с обеих сторон 
зеленые, опушенные коротенькими, снизу более жесткими 
волосами. Цветки обоеполые или однополые, мелкие, на 
длинных цветоножках, часто собраны по 2-5 (реже по 1-2) в рыхлые, 
почти шаровидные, с 5 крупными, сильно сплющенными с боков, 
косточками. Масса 1000 свежесобранных семян (косточек) 
7-16 г, воздушно-сухих — 5-10 г.</s>
